In [ ]:
import pandas as pd
import requests
import urllib3
import os
from dotenv import load_dotenv
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Load environment variables from .env file
load_dotenv()

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': os.getenv('CLIENT_ID'),
    'client_secret': os.getenv('CLIENT_SECRET'),
    'refresh_token': os.getenv('REFRESH_TOKEN'),
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token Granted!\n")

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 10}

desired_activity_order = 1
all_activities = []

param = {'per_page': 1, 'page': desired_activity_order}
# initial request, where we request the first page of activitie
my_dataset = requests.get(activites_url, headers=header, params=param).json()

all_activities=pd.DataFrame(my_dataset)

In [ ]:
all_activities['name'].values[0]

In [ ]:
all_activities

In [ ]:
import plotly.graph_objects as go
import polyline
import numpy as np
import requests
import math

import time
import requests

def get_elevations(coords, batch_size=100, delay=0.5):
    """
    Fetch elevations for multiple coordinates, batching requests and adding delay to avoid rate limit.
    
    coords: list of (lat, lon)
    batch_size: max number of points per request (<=100)
    delay: seconds to wait between batch requests
    returns: list of elevations
    """
    base_url = 'https://api.opentopodata.org/v1/srtm90m'
    elevations = []

    for i in range(0, len(coords), batch_size):
        batch = coords[i:i+batch_size]
        locations_str = "|".join(f"{lat},{lon}" for lat, lon in batch)
        r = requests.get(base_url, params={"locations": locations_str}).json()

        if 'results' in r:
            elevations.extend([res['elevation'] for res in r['results']])
        else:
            raise Exception(r.get('error', 'Unknown error'))

        # Delay to avoid per-second rate limit
        time.sleep(delay)

    return elevations



# Initialize figure
fig = go.Figure()

all_lats = []
all_lons = []

for index, row in all_activities.iterrows():
    polyline_code = row['map']['summary_polyline']
    sport_type = row['sport_type']
    coordinates = polyline.decode(polyline_code, 5)
    
    longitudes = [coordinate[1] for coordinate in coordinates]
    latitudes = [coordinate[0] for coordinate in coordinates]

    # Fetch elevations in a single API call for this polyline
    coords = list(zip(latitudes, longitudes))
    elevation = get_elevations(coords)

    all_lats.extend(latitudes)
    all_lons.extend(longitudes)

    # Add trace for each polyline
    fig.add_trace(go.Scattermap(
        lon=longitudes,
        lat=latitudes,
        mode="lines",
        line=dict(width=2, color='#e91e63'),
        opacity=1,
        showlegend=False
    ))

# Total stats
total_distance_km = all_activities['distance'].sum() / 1000
total_moving_time_min = all_activities['moving_time'].sum() / 60

info_text1 = f"{total_distance_km:.1f} km"
info_text2 = f"{total_moving_time_min:.1f} min"
title = f"Activity title"

fig.update_layout(
    height=700,
    width=600,
    margin=dict(l=0, r=0, t=0, b=0),
    annotations=[
        dict(
            text=title,
            align='center',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.5,
            y=0.98,
            xanchor='center',
            yanchor='top',
            font=dict(size=46, color='#e91e63', family="Courier New, monospace", variant='small-caps'),
            opacity=1,
            bgcolor='rgba(255, 255, 255, 0.6)',
        ),
        dict(
            text=info_text1,
            align='right',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.1,
            y=0.02,
            xanchor='left',
            yanchor='bottom',
            font=dict(size=38, color='#e91e63', family="Courier New, monospace", variant='small-caps'),
            opacity=1,
            bgcolor='rgba(255, 255, 255, 0.6)',
        ),
        dict(
            text=info_text2,
            align='right',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.9,
            y=0.02,
            xanchor='right',
            yanchor='bottom',
            font=dict(size=38, color='#e91e63', family="Courier New, monospace", variant='small-caps'),
            opacity=1,
            bgcolor='rgba(255, 255, 255, 0.6)',
        )
    ]
)

fig.update_layout(
    map_style="https://tiles.stadiamaps.com/styles/alidade_smooth.json",
    map_center_lat=np.mean(all_lats),
    map_center_lon=np.mean(all_lons),
    map_zoom=14.2
)

fig.show()
